In [8]:
import itertools
import copy as c
import pandas as pd
import numpy as np

file1 = pd.read_table(r'..\MSS\MHWstone.txt', encoding="shift_jis", header=None, sep=',')
file2 = pd.read_table(r'..\MSS\MHWskmax.txt', encoding="shift_jis", header=None, sep=',')
file3 = open(r'..\MSS\MHW.txt','r')

lMHW = file3.readlines()

LMHW=[]
for sss in range(len(lMHW)):
    LMHW.append(lMHW[sss][0:-1].split(','))

for sss in range(len(LMHW)):
    try :
        if len(LMHW[sss][0])!=1:
            sentou=LMHW[sss][0]
            while len(LMHW[sss+1][0])==1:
                LMHW[sss+1].insert(0,sentou)
    except:
        continue

file3 = pd.DataFrame(data=LMHW).fillna(0)
file3 = file3[ file3[17] != 0 ].drop([2,9,10,11,12,13,14],axis=1)

count=int(input("検索したいスキル数を入力(3以上推奨、最大8)"))

print(file2[0])

Linputskill=np.array([])
Linputskillname=np.array([])
printcount=1
while printcount<count+1:
    x=int(input("スキル番号を入力してください："))
    Linputskill=np.append(Linputskill,x)
    Linputskillname=np.append(Linputskillname,file2.iloc[x][0].replace(' ', ''))
    printcount+=1
    print(list(Linputskillname))

groupskill1 = file3.groupby(3)
groupskill2 = file3.groupby(5)
groupskill3 = file3.groupby(7)
LMHW=np.empty([1,11])
for sss in range(len(list(Linputskillname))):
    try:
        f1dummy = np.array( groupskill1.get_group(list(Linputskillname)[sss]) )
        LMHW = np.r_[ LMHW , f1dummy ]
        f2dummy = np.array( groupskill2.get_group(list(Linputskillname)[sss]) )
        LMHW = np.r_[ LMHW , f2dummy ]
        f3dummy = np.array( groupskill3.get_group(list(Linputskillname)[sss]) )
        LMHW = np.r_[ LMHW , f3dummy ]
    except:
        continue
print(Linputskillname)
LMHW = np.delete(LMHW, 0, axis=0)
di={"a":3,"b":2,"c":1,"n":0}

LMHW = pd.DataFrame(data=LMHW).replace(di)
#LMHW = LMHW[LMHW.duplicated() == False]

d=file2[0].to_dict()
d=dict( zip( [i.rstrip(" ") for i in d.values()] , d.keys() ) )

LMHW.loc[:,(2,4,6)] = LMHW.loc[:,(2,4,6)].replace(d)
LMHW = LMHW.groupby(1)

LA=np.array(LMHW.get_group("頭"))
LB=np.array(LMHW.get_group("胴"))
LC=np.array(LMHW.get_group("腕"))
LD=np.array(LMHW.get_group("腰"))
LE=np.array(LMHW.get_group("脚"))
LF=np.array(LMHW.get_group("石"))

L =np.r_[LA,LB]
L =np.r_[L ,LC]
L =np.r_[L ,LD]
L =np.r_[L ,LE]
L =np.r_[L ,LF]


A=[i for i in range(LA.shape[0])]
B=[i for i in range(LA.shape[0] , LA.shape[0]+LB.shape[0])]
C=[i for i in range(LA.shape[0]+LB.shape[0] , LA.shape[0]+LB.shape[0]+LC.shape[0])]
D=[i for i in range(LA.shape[0]+LB.shape[0]+LC.shape[0] , LA.shape[0]+LB.shape[0]+LC.shape[0]+LD.shape[0])]
E=[i for i in range(LA.shape[0]+LB.shape[0]+LC.shape[0]+LD.shape[0] , LA.shape[0]+LB.shape[0]+LC.shape[0]+LD.shape[0]+LE.shape[0])]
F=[i for i in range(LA.shape[0]+LB.shape[0]+LC.shape[0]+LD.shape[0]+LE.shape[0] , LA.shape[0]+LB.shape[0]+LC.shape[0]+LD.shape[0]+LE.shape[0]+LF.shape[0])]

Lindex=[]
for _x,_y,_z,_v,_w,_l in itertools.product(A,B,C,D,E,F):
    Lindex.append([_x,_y,_z,_v,_w,_l])

#L[Lindex]が防具名入り、MHWが数値のみ    
MHW = L[Lindex][:,:,2:8]
MHW[MHW == ""] = 0
MHW = MHW.astype(int)

#MHWは三次元、36(防具部位、スキル枠横6)項目ひとまとまり
#MHW

MHW2 = np.array([])
for sss in range(len(Linputskill)):
    E = c.deepcopy(MHW)
    MHW[:,:,1][ MHW[:,:,0] != int(Linputskill[sss]) ] = 0
    MHW[:,:,0] = Linputskill[sss]
    if MHW2 != np.array([]):
        MHW2 = np.r_[MHW2 , MHW[:,:,1][ MHW[:,:,0] == Linputskill[sss] ] ]
    else:
        MHW2 = MHW[:,:,1][ MHW[:,:,0] == Linputskill[sss] ]
    MHW = c.deepcopy(E)
print()
MHW2 = MHW2.reshape(len(list(Linputskill)) , len(list(MHW2))//6//len(list(Linputskill)), 6)
MHW2=np.sum(MHW2, axis=2)
#行がスキル名　列がそのスキルの合計値
#MHW2

#Linputskillから上限値のリスト作成
SKmax=file2.iloc[:,1]
SKmax=SKmax.to_dict()
Linputskill2 = pd.DataFrame(Linputskill)
Linputskill2 = np.array(Linputskill2.replace(SKmax)).astype(int)
#Linputskill2.shape
#MHW2と列数合わせ
Linputskill3=np.tile(Linputskill2,(1,MHW2.shape[1]))
Linputskill=np.array(Linputskill)
#True or False のarray
AYbool = MHW2 <= Linputskill3
AYbool = np.sum(AYbool, axis=0)
MHWindex = np.arange(MHW2.shape[1])
AYbool = MHWindex[AYbool == len(list(Linputskill)) ]
MHW3 = L[Lindex][AYbool]
MHW3[:,:,2:11] = np.where( MHW3[:,:,2:11] == "" ,0 ,MHW3[:,:,2:11])
MHW3[:,:,2:8] = MHW3[:,:,2:8].astype(int)

score = MHW3[:,:,8]+MHW3[:,:,9]+MHW3[:,:,10]
score = np.sum(score, axis=1).reshape(-1,1)
score = score + MHW2.T.sum(axis=1)[AYbool].reshape(-1,1)
scoreindex = np.array( [i for i in range( score.shape[0] )] ).reshape(-1,1)
score = np.r_["1",scoreindex,score]
score = score[ np.argsort(score[:, 1])[::-1] ]
scoreindex = score[:,0].astype("int32")
score = score[:,1]


list(MHW3[scoreindex])


検索したいスキル数を入力(3以上推奨、最大8)3
0        アイテム使用強化 
1              威嚇 
2           運搬の達人 
3         オトモへの采配 
4           ガード強化 
5           ガード性能 
6          回避距離UP 
7            回避性能 
8            回復速度 
9            火事場力 
10           滑走強化 
11        雷属性攻撃強化 
12            雷耐性 
13             感知 
14     貫通弾・竜の一矢強化 
15           気絶耐性 
16         キノコ大好き 
17           強化持続 
18          クライマー 
19            KO術 
20            研究者 
21            広域化 
22             攻撃 
23           剛刃研磨 
24        氷属性攻撃強化 
25            氷耐性 
26          こやし名人 
27             渾身 
28        昆虫標本の達人 
29          採集の達人 
          ...     
88        火属性攻撃強化 
89            火耐性 
90          ひるみ軽減 
91           風圧耐性 
92          笛吹き名人 
93             不屈 
94         フルチャージ 
95             防御 
96      防御力DOWN耐性 
97            砲撃手 
98             砲術 
99        砲弾装填数UP 
100           ボマー 
101        麻痺属性強化 
102          麻痺耐性 
103        麻痺ビン追加 
104           満足感 
105           見切り 
106       水属性攻撃強化 
107           水耐性 
108   

C:\Users\kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
C:\Users\kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


[array([['バルキンβ', '頭', 55, 1, 0, 0, 0, 0, 2, 0, 0],
        ['ドーベルβ', '胴', 22, 2, 0, 0, 0, 0, 3, 0, 0],
        ['ドラケンα', '腕', 105, 2, 22, 1, 0, 0, 2, 2, 0],
        ['オーグβ', '腰', 22, 2, 0, 0, 0, 0, 2, 0, 0],
        ['ディアブロβ', '脚', 55, 2, 0, 0, 0, 0, 1, 1, 0],
        ['攻撃の護石Ⅲ', '石', 22, 3, 0, 0, 0, 0, 0, 0, 0]], dtype=object),
 array([['バルキンβ', '頭', 55, 1, 0, 0, 0, 0, 2, 0, 0],
        ['ドーベルβ', '胴', 22, 2, 0, 0, 0, 0, 3, 0, 0],
        ['ドラケンα', '腕', 105, 2, 22, 1, 0, 0, 2, 2, 0],
        ['オーグβ', '腰', 22, 2, 0, 0, 0, 0, 2, 0, 0],
        ['ドラケンα', '脚', 22, 2, 62, 1, 0, 0, 2, 0, 0],
        ['耐震の護石Ⅱ', '石', 55, 2, 0, 0, 0, 0, 0, 0, 0]], dtype=object),
 array([['リュウα', '頭', 22, 1, 54, 1, 0, 0, 1, 1, 0],
        ['ハーベストα', '胴', 55, 1, 0, 0, 0, 0, 2, 1, 0],
        ['ドラケンα', '腕', 105, 2, 22, 1, 0, 0, 2, 2, 0],
        ['オーグβ', '腰', 22, 2, 0, 0, 0, 0, 2, 0, 0],
        ['ディアブロβ', '脚', 55, 2, 0, 0, 0, 0, 1, 1, 0],
        ['攻撃の護石Ⅲ', '石', 22, 3, 0, 0, 0, 0, 0, 0, 0]], dtype=object),
 array

In [7]:
## A=MHW3[scoreindex][0:100,:,0:2].reshape(100,12)
B=np.r_["1", np.tile(Linputskillname,(MHW2.shape[1],1)) , MHW2.T][AYbool][scoreindex][0:100]
L=np.r_["1",A,B]
S=list(score)[0:100]
for sss in range(len(L)):
    print(S[sss] , " ".join(L[sss]))
print("上位100件を表示")

AxisError: axis 1 is out of bounds for array of dimension 1